# Milestone 2 – Instruction Parser and Code Generation

## Objective
To convert natural language test instructions into structured actions
and generate basic automation-style code.


In [1]:
!pip install langgraph langchain


In [2]:
def instruction_parser(text):
    steps = text.lower().split(",")
    parsed_steps = []

    for step in steps:
        step = step.strip()

        if step.startswith("open"):
            parsed_steps.append({
                "action": "open",
                "target": step.replace("open", "").strip()
            })

        elif "search" in step:
            parsed_steps.append({
                "action": "search",
                "value": step.replace("search for", "").strip()
            })

        elif "click" in step:
            parsed_steps.append({
                "action": "click",
                "target": step.replace("click on", "").strip()
            })

    return parsed_steps


In [3]:
test_input = "Open Google, search for ChatGPT, click on the first result"
instruction_parser(test_input)


[{'action': 'open', 'target': 'google'},
 {'action': 'search', 'value': 'chatgpt'},
 {'action': 'click', 'target': 'the first result'}]

In [4]:
def map_to_commands(parsed_steps):
    return {
        "steps": parsed_steps,
        "total_steps": len(parsed_steps)
    }


In [5]:
mapped_output = map_to_commands(instruction_parser(
    "Open Google, search for ChatGPT, click on the first result"
))
mapped_output


{'steps': [{'action': 'open', 'target': 'google'},
  {'action': 'search', 'value': 'chatgpt'},
  {'action': 'click', 'target': 'the first result'}],
 'total_steps': 3}

In [6]:
def code_generator(commands):
    code_lines = []

    for step in commands["steps"]:
        if step["action"] == "open":
            code_lines.append(f'driver.get("https://{step["target"]}")')

        elif step["action"] == "search":
            code_lines.append(f'search("{step["value"]}")')

        elif step["action"] == "click":
            code_lines.append(f'click("{step["target"]}")')

    return "\n".join(code_lines)


In [7]:
print(code_generator(mapped_output))


driver.get("https://google")
search("chatgpt")
click("the first result")


In [8]:
from langgraph.graph import StateGraph

class State(dict):
    pass


# LangGraph nodes
def parser_node(state):
    state["parsed"] = instruction_parser(state["input_text"])
    return state


def mapper_node(state):
    state["mapped"] = map_to_commands(state["parsed"])
    return state


def generator_node(state):
    state["code"] = code_generator(state["mapped"])
    return state


# Build the graph
graph = StateGraph(State)

graph.add_node("parser", parser_node)
graph.add_node("mapper", mapper_node)
graph.add_node("generator", generator_node)

graph.set_entry_point("parser")
graph.add_edge("parser", "mapper")
graph.add_edge("mapper", "generator")

app = graph.compile()


In [9]:
# Final End-to-End Demonstration (Milestone 2)

test_input = "Open Google, search for ChatGPT, click on the first result"

print("INPUT (Natural Language):")
print(test_input)

print("\nPARSED OUTPUT:")
parsed = instruction_parser(test_input)
print(parsed)

print("\nMAPPED STRUCTURED COMMANDS:")
mapped = map_to_commands(parsed)
print(mapped)

print("\nGENERATED CODE:")
generated_code = code_generator(mapped)
print(generated_code)


INPUT (Natural Language):
Open Google, search for ChatGPT, click on the first result

PARSED OUTPUT:
[{'action': 'open', 'target': 'google'}, {'action': 'search', 'value': 'chatgpt'}, {'action': 'click', 'target': 'the first result'}]

MAPPED STRUCTURED COMMANDS:
{'steps': [{'action': 'open', 'target': 'google'}, {'action': 'search', 'value': 'chatgpt'}, {'action': 'click', 'target': 'the first result'}], 'total_steps': 3}

GENERATED CODE:
driver.get("https://google")
search("chatgpt")
click("the first result")
